In [ ]:
!pip install crewai -qq

In [ ]:
!pip install crewai_tools -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s 

In [ ]:
import os
os.environ["OPENAI_API_KEY"]="your-api-key-here"

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import FileReadTool
import pandas as pd

# Create tool for file reading
read_csv_tool = FileReadTool(file_path='/content/Synthetic_Financial_datasets_log.csv')

# Agents
data_collector = Agent(
    role="Data Collector",
    goal="Load and profile the dataset using pandas, not by hallucinating.",
    backstory="You are responsible for loading and summarizing the real dataset from a CSV file.",
    tools=[read_csv_tool],   # <-- give ability to read files
    verbose=True,
    reasoning=True,
    memory=True
)

pattern_recognizer = Agent(
    role="Pattern Recognizer",
    goal="Detect suspicious transactions using the actual dataset.",
    backstory="You analyze high-value amounts, suspicious transaction types (TRANSFER, CASH_OUT), and balance inconsistencies.",
    tools=[read_csv_tool],
    verbose=True,
    reasoning=True,
    memory=True

)

reporter = Agent(
    role="Fraud Reporter",
    goal="Generate a fraud detection report summarizing anomalies.",
    backstory="You prepare professional reports based on the actual dataset findings.",
    verbose=True,
    reasoning=True,
    memory=True
)

# Tasks
load_task = Task(
    description=(
    "Use FileReadTool to analyze anomalies in batches of 500 rows at a time "
    "from '/content/Synthetic_Financial_datasets_log.csv'. "
    "Focus on suspicious transaction types (TRANSFER, CASH_OUT), very large amounts, "
    "and balance inconsistencies. Summarize anomalies with row indices and amounts."
    ),
    agent=data_collector,
    expected_output="Dataset profile with row count, column names, dtypes, missing values, and 5 real sample rows.",
    input_data={"dataset_path": "PS_20174392719_1491204439457_log.csv"}
)

detect_task = Task(
    description=(
        "Analyze the loaded dataset to identify anomalies: "
        "very high transaction amounts, suspicious types (TRANSFER, CASH_OUT), "
        "and balance inconsistencies. Provide examples with row indices."
    ),
    agent=pattern_recognizer,
    expected_output="A list of detected anomalies with explanations."
)

report_task = Task(
    description="Prepare a structured fraud detection report summarizing anomalies and recommendations.",
    agent=reporter,
    expected_output="Fraud detection report (executive summary + findings + recommendations)."
)

# Crew
crew = Crew(
    agents=[data_collector, pattern_recognizer, reporter],
    tasks=[load_task, detect_task, report_task],
    verbose=True,
    process=Process.sequential,  # The manager oversees the task flow
    planning=True
)

# Run
result = crew.kickoff()
print("\n=== Final Fraud Report ===\n")
print(result)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c93e9ef1-56bb-47f7-8691-9b830b07d103                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 
[2025-09-13 09:51:09][INFO]: Planning the crew execution
[EventBus Error] Handler 'on_task_started' failed for event 'TaskStartedEvent': 'NoneType' object has no attribute 'key'


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 169839d9-232b-4dde-8357-ff61321d93f9                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Fraud Reporter                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Fraud Detection Report                                                                                      │
│                                                                                                                 │
│  ### Executive Summary                                                                                          │
│  The purpose of this report is to summarize detected anomalies in financial transactions to assess potential    │
│  fraud risks. A total of **5 anomalies** were identified, with a focus on transactions classified as            │
│  fraudulent. This report will provide detailed findings of the anomalies detected, actionable recommendations   │
│  for mitigating risks, and insights into transaction behaviors that may indicate fraudulent activity.           │
│                                                                                                                 │
│  ### Findings                                                                                                   │
│                                                                                                                 │
│  #### 1. Anomaly Types Identified                                                                               │
│  - **CASH_OUT Anomalies**                                                                                       │
│    - **Example 1**:                                                                                             │
│      - **Transaction Type**: CASH_OUT                                                                           │
│      - **Transaction Amount**: $181.00                                                                          │
│      - **Origin Account**: C840083671                                                                           │
│      - **Old Balance**: $181.00                                                                                 │
│      - **New Balance**: $0.00                                                                                   │
│      - **Destination Account**: C38997010                                                                       │
│      - **Fraud Status**: Is Fraud                                                                               │
│                                                                                                                 │
│    - **Example 2**:                                                                                             │
│      - **Transaction Type**: CASH_OUT                                                                           │
│      - **Transaction Amount**: $1846.20                                                                         │
│      - **Origin Account**: C2671041864                                                                          │
│      - **Old Balance**: $22,540.00                                                                              │
│      - **New Balance**: $20,693.80                                                                              │
│      - **Destination Account**: C2794781                                                                        │
│      - **Fraud Status**: Is Fraud                                                                               │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 977a3b2a-ab60-49d4-8b3e-0d1f558baa39                                                                     │
│  Agent: Fraud Reporter                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c93e9ef1-56bb-47f7-8691-9b830b07d103                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Fraud Detection Report                                                                        │
│                                                                                                                 │
│  ### Executive Summary                                                                                          │
│  The purpose of this report is to summarize detected anomalies in financial transactions to assess potential    │
│  fraud risks. A total of **5 anomalies** were identified, with a focus on transactions classified as            │
│  fraudulent. This report will provide detailed findings of the anomalies detected, actionable recommendations   │
│  for mitigating risks, and insights into transaction behaviors that may indicate fraudulent activity.           │
│                                                                                                                 │
│  ### Findings                                                                                                   │
│                                                                                                                 │
│  #### 1. Anomaly Types Identified                                                                               │
│  - **CASH_OUT Anomalies**                                                                                       │
│    - **Example 1**:                                                                                             │
│      - **Transaction Type**: CASH_OUT                                                                           │
│      - **Transaction Amount**: $181.00                                                                          │
│      - **Origin Account**: C840083671                                                                           │
│      - **Old Balance**: $181.00                                                                                 │
│      - **New Balance**: $0.00                                                                                   │
│      - **Destination Account**: C38997010                                                                       │
│      - **Fraud Status**: Is Fraud                                                                               │
│                                                                                                                 │
│    - **Example 2**:                                                                                             │
│      - **Transaction Type**: CASH_OUT                                                                           │
│      - **Transaction Amount**: $1846.20                                                                         │
│      - **Origin Account**: C2671041864                                                                          │
│      - **Old Balance**: $22,540.00                                                                              │
│      - **New Balance**: $20,693.80                                                                              │
│      - **Destination Account**: C2794781                                                                        │
│      - **Fraud Status**: Is Fraud                     


=== Final Fraud Report ===

## Fraud Detection Report

### Executive Summary
The purpose of this report is to summarize detected anomalies in financial transactions to assess potential fraud risks. A total of **5 anomalies** were identified, with a focus on transactions classified as fraudulent. This report will provide detailed findings of the anomalies detected, actionable recommendations for mitigating risks, and insights into transaction behaviors that may indicate fraudulent activity.

### Findings

#### 1. Anomaly Types Identified
- **CASH_OUT Anomalies**
  - **Example 1**: 
    - **Transaction Type**: CASH_OUT 
    - **Transaction Amount**: $181.00 
    - **Origin Account**: C840083671 
    - **Old Balance**: $181.00 
    - **New Balance**: $0.00 
    - **Destination Account**: C38997010 
    - **Fraud Status**: Is Fraud
   
  - **Example 2**: 
    - **Transaction Type**: CASH_OUT 
    - **Transaction Amount**: $1846.20 
    - **Origin Account**: C2671041864 
    - **Old Balanc